<a href="https://colab.research.google.com/github/Gaurav-Ramachandra/Sem5-BIS_Lab/blob/main/Expt2%2024-10%3A%20Particle%20Swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Particle Swarm

import numpy as np

# Define the objective function (to be minimized)
def objective_function(position):
    # Example: Sphere function (minimizing sum of squares)
    return np.sum(position ** 2)

# Initialize particles' positions and velocities
def initialize_particles(num_particles, dimensions, lower_bound, upper_bound):
    particles = np.random.uniform(low=lower_bound, high=upper_bound, size=(num_particles, dimensions))
    velocities = np.random.uniform(low=-1, high=1, size=(num_particles, dimensions))
    return particles, velocities

# Update velocity based on inertia, cognitive, and social components
def update_velocity(velocity, particle_position, personal_best, global_best, inertia_weight, cognitive_coeff, social_coeff):
    cognitive_component = cognitive_coeff * np.random.rand() * (personal_best - particle_position)
    social_component = social_coeff * np.random.rand() * (global_best - particle_position)
    new_velocity = inertia_weight * velocity + cognitive_component + social_component
    return new_velocity

# Main PSO function
def particle_swarm_optimization(objective_function, num_particles, dimensions, num_iterations, inertia_weight, cognitive_coeff, social_coeff, lower_bound, upper_bound):
    # Step 1: Initialize particles and velocities
    particles, velocities = initialize_particles(num_particles, dimensions, lower_bound, upper_bound)

    # Initialize personal best positions and global best position
    personal_best_positions = np.copy(particles)
    personal_best_scores = np.apply_along_axis(objective_function, 1, personal_best_positions)
    global_best_position = personal_best_positions[np.argmin(personal_best_scores)]

    # Main optimization loop
    for iteration in range(num_iterations):
        # Step 2: Evaluate fitness (objective function) for each particle
        for i in range(num_particles):
            fitness = objective_function(particles[i])

            # Update personal best if current position is better
            if fitness < personal_best_scores[i]:
                personal_best_positions[i] = particles[i]
                personal_best_scores[i] = fitness

            # Update global best if current position is better
            if fitness < np.min(personal_best_scores):
                global_best_position = particles[i]

        # Step 3: Update velocities and positions
        for i in range(num_particles):
            velocities[i] = update_velocity(velocities[i], particles[i], personal_best_positions[i], global_best_position, inertia_weight, cognitive_coeff, social_coeff)
            particles[i] = particles[i] + velocities[i]  # Update position

        # (Optional) Print progress
        print(f"Iteration {iteration+1}/{num_iterations}, Global Best Fitness: {objective_function(global_best_position)}")

    # Step 4: Return the best solution found
    return global_best_position, objective_function(global_best_position)

# Parameters
num_particles = 30       # Number of particles
dimensions = 2           # Dimensions of the problem (e.g., 2D space)
num_iterations = 100     # Number of iterations
inertia_weight = 0.7     # Inertia weight
cognitive_coeff = 1.5    # Cognitive (personal best) coefficient
social_coeff = 1.5       # Social (global best) coefficient
lower_bound = -10        # Lower bound of the search space
upper_bound = 10         # Upper bound of the search space

# Run PSO
best_position, best_value = particle_swarm_optimization(objective_function, num_particles, dimensions, num_iterations, inertia_weight, cognitive_coeff, social_coeff, lower_bound, upper_bound)

print(f"Best position: {best_position}")
print(f"Best value (minimum): {best_value}")

Iteration 1/100, Global Best Fitness: 4.8091342803262505
Iteration 2/100, Global Best Fitness: 3.599552152447158
Iteration 3/100, Global Best Fitness: 2.9150348243113164
Iteration 4/100, Global Best Fitness: 2.515345873692193
Iteration 5/100, Global Best Fitness: 2.2745054660060307
Iteration 6/100, Global Best Fitness: 2.124998690921856
Iteration 7/100, Global Best Fitness: 2.029693888408042
Iteration 8/100, Global Best Fitness: 1.9675619972704756
Iteration 9/100, Global Best Fitness: 1.9263145940790096
Iteration 10/100, Global Best Fitness: 1.8985414229413502
Iteration 11/100, Global Best Fitness: 1.8796392085822085
Iteration 12/100, Global Best Fitness: 1.8666717711950471
Iteration 13/100, Global Best Fitness: 1.8577239802295105
Iteration 14/100, Global Best Fitness: 1.8515239400043182
Iteration 15/100, Global Best Fitness: 1.8472149844375187
Iteration 16/100, Global Best Fitness: 1.8442139411102683
Iteration 17/100, Global Best Fitness: 1.8421206713102525
Iteration 18/100, Global Be

In [ ]:
# Particle Swarm for travelling salesman problem

import numpy as np
import random

# Sample data: Distance matrix between cities (example with 5 cities)
distance_matrix = np.array([
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 20],
    [20, 25, 30, 0, 15],
    [25, 30, 20, 15, 0]
])
num_cities = len(distance_matrix)

# PSO parameters
num_particles = 30
num_iterations = 100
w = 0.5          # Inertia weight
c1 = 1.5         # Cognitive component
c2 = 1.5         # Social component

# Calculate total distance of a route
def route_distance(route):
    distance = sum([distance_matrix[route[i], route[i+1]] for i in range(len(route) - 1)])
    distance += distance_matrix[route[-1], route[0]]  # Returning to the starting city
    return distance

# Initialize particles
def initialize_particles():
    particles = []
    velocities = []
    personal_best = []
    personal_best_scores = []
    global_best_route = None
    global_best_score = float('inf')

    for _ in range(num_particles):
        route = list(np.random.permutation(num_cities))
        score = route_distance(route)
        particles.append(route)
        velocities.append([random.sample(range(num_cities), 2) for _ in range(3)])  # Random swap pairs as velocity
        personal_best.append(route)
        personal_best_scores.append(score)

        if score < global_best_score:
            global_best_score = score
            global_best_route = route

    return particles, velocities, personal_best, personal_best_scores, global_best_route, global_best_score

# Update velocity using swaps (permutation-based velocity update)
def update_velocity(velocity, particle, personal_best, global_best):
    new_velocity = []

    # Cognitive component
    for _ in range(int(c1 * random.random())):
        swap = random.sample(range(num_cities), 2)
        if personal_best[swap[0]] != particle[swap[0]]:
            new_velocity.append(swap)

    # Social component
    for _ in range(int(c2 * random.random())):
        swap = random.sample(range(num_cities), 2)
        if global_best[swap[0]] != particle[swap[0]]:
            new_velocity.append(swap)

    # Inertia component (keep part of current velocity)
    new_velocity.extend(random.sample(velocity, int(w * len(velocity))))

    return new_velocity

# Apply velocity by performing swaps on the particle's route
def apply_velocity(route, velocity):
    new_route = route[:]
    for swap in velocity:
        new_route[swap[0]], new_route[swap[1]] = new_route[swap[1]], new_route[swap[0]]
    return new_route

# Main PSO for TSP
def pso_tsp():
    particles, velocities, personal_best, personal_best_scores, global_best_route, global_best_score = initialize_particles()

    for iteration in range(num_iterations):
        for i in range(num_particles):
            # Apply velocity to get a new route
            new_route = apply_velocity(particles[i], velocities[i])
            new_score = route_distance(new_route)

            # Update personal best
            if new_score < personal_best_scores[i]:
                personal_best[i] = new_route
                personal_best_scores[i] = new_score

            # Update global best
            if new_score < global_best_score:
                global_best_route = new_route
                global_best_score = new_score

            # Update velocity
            velocities[i] = update_velocity(velocities[i], particles[i], personal_best[i], global_best_route)

            # Update particle's position
            particles[i] = new_route

        # Print progress
        print(f"Iteration {iteration}: Best Distance = {global_best_score}")

    return global_best_route, global_best_score

# Run PSO for TSP
best_route, best_distance = pso_tsp()
print("Best Route:", best_route)
print("Best Distance:", best_distance)


Iteration 0: Best Distance = 85
Iteration 1: Best Distance = 85
Iteration 2: Best Distance = 85
Iteration 3: Best Distance = 85
Iteration 4: Best Distance = 85
Iteration 5: Best Distance = 85
Iteration 6: Best Distance = 85
Iteration 7: Best Distance = 85
Iteration 8: Best Distance = 85
Iteration 9: Best Distance = 85
Iteration 10: Best Distance = 85
Iteration 11: Best Distance = 85
Iteration 12: Best Distance = 85
Iteration 13: Best Distance = 85
Iteration 14: Best Distance = 85
Iteration 15: Best Distance = 85
Iteration 16: Best Distance = 85
Iteration 17: Best Distance = 85
Iteration 18: Best Distance = 85
Iteration 19: Best Distance = 85
Iteration 20: Best Distance = 85
Iteration 21: Best Distance = 85
Iteration 22: Best Distance = 85
Iteration 23: Best Distance = 85
Iteration 24: Best Distance = 85
Iteration 25: Best Distance = 85
Iteration 26: Best Distance = 85
Iteration 27: Best Distance = 85
Iteration 28: Best Distance = 85
Iteration 29: Best Distance = 85
Iteration 30: Best D